# Índice Invertido y Modelo Booleano

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jeffr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Tokenizar y eliminar stopwords

In [52]:
with open("docs/stoplist.txt", encoding="utf-8") as file:
    stoplist = [line.rstrip().lower() for line in file]

def preProcesamiento(texto):
    words = []
    # 1- tokenizar: Los tokens son las palabras que conforman el texto se colocan en una lista. Las palabras pueden contener signos de puntuación.
    tokens = nltk.word_tokenize(texto) 

    # 2- eliminar signos: Solo se toman las palabras que contienen letras y números y se convierten a minúsculas.
    for token in tokens:
        if token.isalnum():
            words.append(token.lower())

    # 3- eliminar stopwords, agregar mas stopwords: Los stopwords son palabras que no aportan significado al texto, como artículos, preposiciones, etc.
    words = [word for word in words if word not in stoplist]

    # 4- Aplicar la reduccion de palabras (lexema) y aplicar la normalizacion de palabras con SnowballStemmer: Se convierten las palabras a su raíz o lexema.
    stemmer = nltk.SnowballStemmer("spanish")
    words = [stemmer.stem(word) for word in words]
    # print("Words after stemming: ", words)

    return words

# test
preprocesedWords = preProcesamiento("hola a todos! aqui programando en python, que divertido es esto!")
print(preprocesedWords)


['hol', 'program', 'python', 'divert']


### Construir Índice Invertido

In [71]:
def indexar(books):    
    indice = dict()

    # for file_name in books:
    #     print("Indexando: ", file_name, "...")
    #     File = open(file_name, encoding="utf-8")
    #     texto = File.read().rstrip();
    #     # 1- preprocesar: aplicar preprocesamiento al texto para obtener los lexemas de las palabras
    #     words = preProcesamiento(texto)

    #     # 2- agregar al indice los keywords a manera de diccionario. Se agrega el nombre del archivo a la lista de archivos donde se encuentra la palabra.
    #     for word in words: # por cada lexema en el texto
    #         if word in indice: # si la palabra ya esta en el indice
    #             if file_name not in indice[word]: # si el archivo no esta en la lista de archivos donde se encuentra la palabra
    #                 indice[word].append(file_name) # se agrega el archivo a la lista de archivos donde se encuentra la palabra
    #         else:
    #             indice[word] = [] # si la palabra no esta en el indice, se agrega al indice con una lista vacia
    #             indice[word].append(file_name) # se agrega el archivo a la lista de archivos donde se encuentra la palabra
    

    for i in range(len(books)):
        print("Indexando: ", books[i], "...")
        File = open(books[i], encoding="utf-8")
        texto = File.read().rstrip()

        words = preProcesamiento(texto)

        for word in words:
            if word in indice:
                if i not in indice[word]:
                    indice[word].append(i)
            else: 
                indice[word] = []
                indice[word].append(i)
    
    return indice

# test
books = ["docs/libro1.txt", "docs/libro2.txt", "docs/libro3.txt", "docs/libro4.txt", "docs/libro5.txt", "docs/libro6.txt"]
Indice = indexar(books)
print(sorted(Indice.items()))

Indexando:  docs/libro1.txt ...
Indexando:  docs/libro2.txt ...
Indexando:  docs/libro3.txt ...
Indexando:  docs/libro4.txt ...
Indexando:  docs/libro5.txt ...
Indexando:  docs/libro6.txt ...
[('111º', [0]), ('abaj', [5]), ('abiert', [4]), ('abism', [1, 2]), ('acab', [0, 3, 4, 5]), ('acamp', [4]), ('accidental', [0]), ('accion', [2]), ('acebed', [1]), ('acerc', [1]), ('acompañ', [0, 1, 4, 5]), ('aconsej', [2]), ('acontec', [1, 2]), ('actos', [2]), ('acud', [0, 1, 4, 5]), ('adentr', [0]), ('afortun', [3]), ('agot', [5]), ('agu', [1]), ('aguil', [5]), ('air', [5]), ('alcanz', [1]), ('alde', [0]), ('alegr', [5]), ('alert', [0, 3, 4]), ('alli', [0, 1, 4, 5]), ('altern', [4]), ('amenaz', [3]), ('amig', [0, 1, 2]), ('amo', [1, 3, 5]), ('amon', [0, 1]), ('anduin', [0, 1, 4]), ('anill', [0, 1, 3, 5]), ('aniquil', [1]), ('antigu', [1]), ('anul', [5]), ('apag', [3]), ('aparec', [1, 3]), ('aparent', [2, 3]), ('apart', [1]), ('apen', [1]), ('apoder', [5]), ('apoy', [4]), ('apres', [2]), ('apresur'

### Aplicar consultas

In [87]:
def AND(list1, list2):
    result = []
    res_books = []
    i = j = 0

    while i < len(list1) and j < len(list2):
        print("Comparando: ", list1[i] < list2[j])
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            j += 1

    for i in result:
        res_books.append(books[i])

    return res_books

def OR(list1, list2):
    result = []
    res_books = []
    i = j = 0

    while i < len(list1) and j < len(list2):
        print("Comparando: ", list1[i], list2[j])
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1

    for i in result:
        res_books.append(books[i])

    return res_books

def AND_NOT(list1, list2):
    result = []
    res_books = []
    i = j = 0
    
    while i < len(list1) and j < len(list2):
        print("Comparando: ", list1[i], list2[j])
        if list1[i] == list2[j]:
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1

    for i in result:
        res_books.append(books[i])

    return res_books

# Recibe una palabra
def L(word):
    word = preProcesamiento(word)[0] # Retorna el lexema de la palabra
    print("Palabra:", word, "- Archivos: ", Indice[word])
    return Indice[word] # Retorna la lista de archivos donde se encuentra la palabra


In [89]:
result = AND(L("frodo"), L("anillo"))
print("Los libros son:")
print(result)
#['libro1.txt', 'libro2.txt', 'libro4.txt', 'libro6.txt']

Palabra: frod - Archivos:  [0, 1, 2, 3, 4, 5]
Palabra: anill - Archivos:  [0, 1, 3, 5]
Comparando:  False
Comparando:  False
Comparando:  True
Comparando:  False
Comparando:  True
Comparando:  False
Los libros son:
['docs/libro1.txt', 'docs/libro2.txt', 'docs/libro4.txt', 'docs/libro6.txt']


In [82]:
result = OR(L("anillo"), L("frodo"))
print("Los libros son:")
print(result)

Palabra: anill - Archivos:  [0, 1, 3, 5]
Palabra: frod - Archivos:  [0, 1, 2, 3, 4, 5]
Comparando:  0 0
Comparando:  1 1
Comparando:  3 2
Comparando:  3 3
Comparando:  5 4
Comparando:  5 5
Los libros son:
['docs/libro1.txt', 'docs/libro2.txt', 'docs/libro3.txt', 'docs/libro4.txt', 'docs/libro5.txt', 'docs/libro6.txt']


In [88]:
result = AND_NOT(L("anillo"), L("frodo"))
print("Los libros son:")
print(result)

Palabra: anill - Archivos:  [0, 1, 3, 5]
Palabra: frod - Archivos:  [0, 1, 2, 3, 4, 5]
Comparando:  0 0
Comparando:  1 1
Comparando:  3 2
Comparando:  3 3
Comparando:  5 4
Comparando:  5 5
Los libros son:
['docs/libro3.txt', 'docs/libro5.txt']


In [119]:
#CONSULTA 1 con comunidad, frondo and gondor
result = AND_NOT(AND(L("comunidad"), L("frodo")), L("unico"))
print("Los libros son:")
print(result)

#CONSULTA 2 con abismo or acaba and not acampar
result = AND_NOT(OR(L("abismo"), L("acaba")), L("acampar"))
print("Los libros son:")
print(result)

#CONSULTA 3 con acontecimientos and actos or alerta
result = OR(AND(L("acontecimientos"), L("actos")), L("alerta"))
print("Los libros son:")
print(result)



Los libros son:
[]
Los libros son:
['libro1.txt', 'libro2.txt', 'libro3.txt']
Los libros son:
['libro1.txt', 'libro3.txt']
